Author: Sofia Ingersoll


Title: WK6 SQL Q1


Date: 2024/05/14



#### Step 1

Create a query that will return the distinct species for which there is egg data (not all species and not all nests have egg data), so that you can then loop over those species. Your query should return two columns, species code and scientific name. Please order the results in alphabetic order of scientific name.


In [7]:
import pandas as pd
import duckdb

In [8]:
# Connect to the DuckDB database
conn = duckdb.connect(database='database.db')  # Replace 'your_database.db' with your actual database file

cur = conn.cursor()

In [18]:
# I can never remember the column names of the tables
# let's take a peek
cur.execute("DESCRIBE Bird_nests")
cur.fetchall()

[('Book_page', 'VARCHAR', 'YES', None, None, None),
 ('Year', 'INTEGER', 'NO', None, None, None),
 ('Site', 'VARCHAR', 'NO', None, None, None),
 ('Nest_ID', 'VARCHAR', 'NO', 'PRI', None, None),
 ('Species', 'VARCHAR', 'NO', None, None, None),
 ('Observer', 'VARCHAR', 'YES', None, None, None),
 ('Date_found', 'DATE', 'NO', None, None, None),
 ('how_found', 'VARCHAR', 'YES', None, None, None),
 ('Clutch_max', 'INTEGER', 'YES', None, None, None),
 ('floatAge', 'FLOAT', 'YES', None, None, None),
 ('ageMethod', 'VARCHAR', 'YES', None, None, None)]

In [16]:
# let's take a peek at Bird eggs table
cur.execute("DESCRIBE Bird_eggs")
cur.fetchall()

[('Book_page', 'VARCHAR', 'YES', None, None, None),
 ('Year', 'INTEGER', 'NO', None, None, None),
 ('Site', 'VARCHAR', 'NO', None, None, None),
 ('Nest_ID', 'VARCHAR', 'NO', 'PRI', None, None),
 ('Egg_num', 'INTEGER', 'NO', 'PRI', None, None),
 ('Length', 'FLOAT', 'NO', None, None, None),
 ('Width', 'FLOAT', 'NO', None, None, None)]

In [17]:
cur.execute("DESCRIBE Species")
cur.fetchall()

[('Code', 'VARCHAR', 'NO', 'PRI', None, None),
 ('Common_name', 'VARCHAR', 'NO', 'UNI', None, None),
 ('Scientific_name', 'VARCHAR', 'YES', None, None, None),
 ('Relevance', 'VARCHAR', 'YES', None, None, None)]

In [26]:
species_query = """
SELECT DISTINCT S.Code, S.Scientific_name
FROM Species AS S
JOIN Bird_nests AS BN ON S.Code = BN.Species
JOIN Bird_eggs AS BE ON BN.Nest_ID = BE.Nest_ID
WHERE BE.Length IS NOT NULL AND BE.Width IS NOT NULL
ORDER BY S.Scientific_name;
"""

# Read data into Pandas DataFrame
cur.execute(species_query)
cur.fetchall()


[('rutu', 'Arenaria interpres'),
 ('dunl', 'Calidris alpina'),
 ('wrsa', 'Calidris fuscicollis'),
 ('sepl', 'Charadrius semipalmatus'),
 ('reph', 'Phalaropus fulicarius'),
 ('amgp', 'Pluvialis dominica'),
 ('bbpl', 'Pluvialis squatarola')]

#### Step 2

After you’ve connected to the database and created a cursor c, iterate over the species like so:
```{SQL}
species_query = """SELECT Code, Scientific_name FROM..."""
for row in c.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # query egg data for that species (step 3)
    # compute statistics and print results (step 4)


```


In [29]:
species_query = """
SELECT DISTINCT S.Code, S.Scientific_name
FROM Species AS S
JOIN Bird_nests AS BN ON S.Code = BN.Species
JOIN Bird_eggs AS BE ON BN.Nest_ID = BE.Nest_ID
WHERE BE.Length IS NOT NULL AND BE.Width IS NOT NULL
ORDER BY S.Scientific_name;
"""
for row in conn.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # Let's add some specificity to our iteration!
    print(f"Species Code: {species_code}, Scientific Name: {scientific_name}")

Species Code: rutu, Scientific Name: Arenaria interpres
Species Code: dunl, Scientific Name: Calidris alpina
Species Code: wrsa, Scientific Name: Calidris fuscicollis
Species Code: sepl, Scientific Name: Charadrius semipalmatus
Species Code: reph, Scientific Name: Phalaropus fulicarius
Species Code: amgp, Scientific Name: Pluvialis dominica
Species Code: bbpl, Scientific Name: Pluvialis squatarola


#### Step 3

You will need to construct a query that gathers egg data for a given species, one species at a time; the species code will be a parameter to that query. You can compute the formula

in SQL or in Pandas. For simplicity, SQL is suggested:
```
egg_query = """SELECT Width*Width*Length AS Volume FROM..."""
```

Within the loop, you will want to execute the query on the current species in the loop iteration. You may use the Pandas function pd.read_sql to do so and also directly load the results into a dataframe:
```
df = pd.read_sql(egg_query, conn, ...)
```

Do a help(pd.read_sql) to figure out how to pass parameters to a query.

You may get a bunch of warnings from Pandas about how it “only supports SQLAlchemy…”. Just ignore them. (Sorry about that.)

In [83]:
# establishing df column names in first line
egg_query = """
SELECT S.Code AS Species_Code, S.Scientific_name AS Scientific_Name,
       AVG(Width * Width * Length) AS Volume
FROM Bird_eggs AS BE
JOIN Bird_nests AS BN ON BE.Nest_ID = BN.Nest_ID
JOIN Species AS S ON BN.Species = S.Code
WHERE BE.Length IS NOT NULL AND BE.Width IS NOT NULL
GROUP BY Species_Code, Scientific_Name
ORDER BY S.Scientific_name
"""
species_query = """
SELECT DISTINCT S.Code, S.Scientific_name
FROM Species AS S
JOIN Bird_nests AS BN ON S.Code = BN.Species
JOIN Bird_eggs AS BE ON BN.Nest_ID = BE.Nest_ID
WHERE BE.Length IS NOT NULL AND BE.Width IS NOT NULL
GROUP BY S.Code, S.Scientific_name, Width, Length
ORDER BY S.Scientific_name
"""
for row in conn.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    # Execute the egg query for the current species
    egg_data = conn.execute(egg_query).fetchdf()
    
    # Load results into a Pandas DataFrame
    egg_df = pd.DataFrame(egg_data)

In [84]:
# let's confirm our output is as expected!
egg_data

,Species_Code,Scientific_Name,Volume
0,rutu,Arenaria interpres,34557.250255
1,dunl,Calidris alpina,22492.769886
2,wrsa,Calidris fuscicollis,19310.249314
3,sepl,Charadrius semipalmatus,18912.809756
4,reph,Phalaropus fulicarius,16127.301270
5,amgp,Pluvialis dominica,54515.986328
6,bbpl,Pluvialis squatarola,64888.017361


#### Step 4

Finally, and still within your loop, you’ll want to compute statistics and print out the results:

```
cv = round(df.Volume.std()/df.Volume.mean()*100, 2)
print(f"{scientific_name} {cv}%")
```

In [82]:
# Initialize a dictionary to store statistics for each species
species_stats = {}

for row in conn.execute(species_query).fetchall():  # DuckDB lame-o workaround
    species_code = row[0]
    scientific_name = row[1]
    
    # Execute the egg query for the current species
    egg_query_for_species = f"""
    SELECT Width * Width * Length AS Volume
    FROM Bird_eggs AS BE
    JOIN Bird_nests AS BN ON BE.Nest_ID = BN.Nest_ID
    WHERE BN.Species = '{species_code}' AND BE.Length IS NOT NULL AND BE.Width IS NOT NULL
    """
    egg_data = conn.execute(egg_query_for_species).fetchdf()
    
    # Load results into a Pandas DataFrame
    egg_df = pd.DataFrame(egg_data)

    # Compute statistics
    if not egg_df.empty:
        mean_volume = egg_df['Volume'].mean()
        std_volume = egg_df['Volume'].std()
        cv = round(std_volume / mean_volume * 100, 2)
    else:
        cv = None
    
    # Store the statistics for the current species
    species_stats[scientific_name] = cv

# Print out the statistics for each species
for species, cv in species_stats.items():
    if cv is not None:
        print(f"{species}: {cv}%")


Arenaria interpres: 21.12%
Calidris alpina: 5.46%
Calidris fuscicollis: 16.77%
Charadrius semipalmatus: 8.99%
Phalaropus fulicarius: 4.65%
Pluvialis dominica: 19.88%
Pluvialis squatarola: 6.94%


Let's close out our session!

In [93]:
duckdb.close(conn)